In [64]:
# Tottenham,Newcastle
# Aston Villa,Leicester
# Bournemouth,Everton
# Crystal Palace,Chelsea
# Manchester City,West Ham
# Southampton,Brentford
# Brighton,Arsenal
# Fulham,Ipswich
# Liverpool,Manchester United
# Wolves,Nottingham Forest

In [1]:
import requests
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
gw = 28

fixtures = requests.get(f'https://fantasy.premierleague.com/api/fixtures/?event={gw}').json()
boots_ = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()

fixture_dets =  [{'team_a': fix['team_a'], 'team_h': fix['team_h'], 'h_fdr': fix['team_h_difficulty'], 'a_fdr': fix['team_a_difficulty'] } for fix in fixtures]
fix_df = pd.DataFrame(fixture_dets)

team_ids = [{'team_code': team['code'], 'team_name': team['name'], 'team_id': team['id']} for team in boots_['teams']]
id_df = pd.DataFrame(team_ids)

def add_team_name(row):
    away = row['team_a']
    home = row['team_h']

    home_team_row = id_df[id_df['team_id'] == home]['team_name'].values
    away_team_row = id_df[id_df['team_id'] == away]['team_name'].values

    team_home = home_team_row[0] if home_team_row.size > 0 else 'None'
    team_away = away_team_row[0] if away_team_row.size > 0 else 'None'

    if(team_home == 'Man Utd'):
        team_home = 'Manchester United'
    elif(team_away == 'Man Utd'):
        team_away = 'Manchester United'

    if(team_home == 'Man City'):
        team_home = 'Manchester City'
    elif(team_away == 'Man City'):
        team_away = 'Manchester City'

    if(team_home == 'Wolves'):
        team_home = 'Wolverhampton Wanderers'
    elif(team_away == 'Wolves'):
        team_away = 'Wolverhampton Wanderers'

    if(team_home == "Nott'm Forest"):
        team_home = "Nottingham Forest"
    elif(team_away == "Nott'm Forest"):
        team_away = "Nottingham Forest"

    if(team_home == "Spurs"):
        team_home = "Tottenham"
    elif(team_away == "Spurs"):
        team_away = "Tottenham"

    if(team_home == "Newcastle"):
        team_home = "Newcastle United"
    elif(team_away == "Newcastle"):
        team_away = "Newcastle United"


    return pd.Series([team_home, team_away])


fix_df[['team_home', 'team_away']] = fix_df.apply(add_team_name, axis=1)

fix_df.to_csv(f'./fpl_team_ids_{gw}.csv', index=False)


In [3]:
pd.read_csv(f'./fpl_team_ids_{gw}.csv')

,team_a,team_h,h_fdr,a_fdr,team_home,team_away
0,13,16,4,4,Nottingham Forest,Manchester City
1,9,5,3,3,Brighton,Fulham
2,10,7,2,3,Crystal Palace,Ipswich
3,17,12,2,5,Liverpool,Southampton
4,2,4,3,3,Brentford,Aston Villa
5,8,20,2,2,Wolverhampton Wanderers,Everton
6,11,6,2,4,Chelsea,Leicester
7,3,18,3,3,Tottenham,Bournemouth
8,1,14,4,3,Manchester United,Arsenal
9,15,19,3,2,West Ham,Newcastle United


In [4]:
premLeague = "https://sports.williamhill.com/betting/en-gb/football/competitions/OB_TY295/English-Premier-League/matches/OB_MGMB/Match-Betting"

driver = webdriver.Firefox()
driver.get(premLeague)
matches = driver.find_elements(By.CSS_SELECTOR, "article.sp-o-market--default")[0:10]
details = pd.DataFrame({'h_team':[], 'a_team':[], 'WHH':[], 'WHD':[], 'WHA':[]})

# Get links for the matches
# [[match, link]]
for match in matches:
    # matches_links.append([match.text, match.get_attribute('href')])
    # Extract teams
    teams = match.find_element(By.CSS_SELECTOR, 'main.sp-o-market__title span').text
    odds_els = match.find_elements(By.CSS_SELECTOR, 'section.sp-o-market__buttons .sp-betbutton > span')

    # Extract odds
    # odds = [round(int(btn.text.split('/')[0]) / int(btn.text.split('/')[1]) +1, 2)   for btn in match.find_elements(By.CSS_SELECTOR, 'section.sp-o-market__buttons .sp-betbutton > span')]

    odds = []
    for btn in odds_els:
        if btn.text == 'EVS':
            odds.append(round(2.0, 2))
        else:
            odds.append(round(int(btn.text.split('/')[0]) / int(btn.text.split('/')[1]) +1, 2))
    h_odds = 1/odds[0]
    d_odds = 1/odds[1]
    a_odds = 1/odds[2]

    sum_odd_probs = h_odds + d_odds + a_odds
    match_details = {
        'h_team': [teams.split(' v ')[0]],
        'a_team': [teams.split(' v ')[1]],
        'WHH': [round(h_odds/sum_odd_probs, 3)],
        'WHD': [round(d_odds/sum_odd_probs, 3)],
        'WHA': [round(a_odds/sum_odd_probs, 3)]
        }
    match_dets_df = pd.DataFrame(match_details)
    details = pd.concat([details, match_dets_df], ignore_index=True)
    # print(odds)
    # print({'Home': teams.split(' v ')[0], 'Away': teams.split(' v ')[1], 'WHH': odds[0], 'WHD': odds[1], 'WHA': odds[2]  })


driver.close()


details

,h_team,a_team,WHH,WHD,WHA
0,Nottingham Forest,Man City,0.269,0.248,0.483
1,Brighton,Fulham,0.470,0.261,0.269
2,Crystal Palace,Ipswich,0.649,0.208,0.144
3,Liverpool,Southampton,0.863,0.087,0.050
4,Brentford,Aston Villa,0.400,0.254,0.346
5,Wolves,Everton,0.394,0.291,0.315
6,Chelsea,Leicester,0.783,0.145,0.072
7,Tottenham,Bournemouth,0.395,0.243,0.362
8,Man Utd,Arsenal,0.224,0.268,0.508
9,West Ham,Newcastle,0.288,0.250,0.463


In [5]:

# Manchester City,
# Manchester United
def update_name(row):
    if(row['h_team'] == 'Man Utd'):
        row['h_team'] = 'Manchester United'
    elif(row['a_team'] == 'Man Utd'):
        row['a_team'] = 'Manchester United'

    if(row['h_team'] == 'Man City'):
        row['h_team'] = 'Manchester City'
    elif(row['a_team'] == 'Man City'):
        row['a_team'] = 'Manchester City'

    if(row['h_team'] == 'Wolves'):
        row['h_team'] = 'Wolverhampton Wanderers'
    elif(row['a_team'] == 'Wolves'):
        row['a_team'] = 'Wolverhampton Wanderers'

    if(row['h_team'] == 'Newcastle'):
        row['h_team'] = 'Newcastle United'
    elif(row['a_team'] == 'Newcastle'):
        row['a_team'] = 'Newcastle United'
    return row


details_ = details.apply(update_name, axis=1)
details_

,h_team,a_team,WHH,WHD,WHA
0,Nottingham Forest,Manchester City,0.269,0.248,0.483
1,Brighton,Fulham,0.470,0.261,0.269
2,Crystal Palace,Ipswich,0.649,0.208,0.144
3,Liverpool,Southampton,0.863,0.087,0.050
4,Brentford,Aston Villa,0.400,0.254,0.346
5,Wolverhampton Wanderers,Everton,0.394,0.291,0.315
6,Chelsea,Leicester,0.783,0.145,0.072
7,Tottenham,Bournemouth,0.395,0.243,0.362
8,Manchester United,Arsenal,0.224,0.268,0.508
9,West Ham,Newcastle United,0.288,0.250,0.463


In [6]:

details_.to_csv(f'./odds_{gw}.csv', index=False)

In [7]:
pd.read_csv(f'./odds_{gw}.csv')


,h_team,a_team,WHH,WHD,WHA
0,Nottingham Forest,Manchester City,0.269,0.248,0.483
1,Brighton,Fulham,0.470,0.261,0.269
2,Crystal Palace,Ipswich,0.649,0.208,0.144
3,Liverpool,Southampton,0.863,0.087,0.050
4,Brentford,Aston Villa,0.400,0.254,0.346
5,Wolverhampton Wanderers,Everton,0.394,0.291,0.315
6,Chelsea,Leicester,0.783,0.145,0.072
7,Tottenham,Bournemouth,0.395,0.243,0.362
8,Manchester United,Arsenal,0.224,0.268,0.508
9,West Ham,Newcastle United,0.288,0.250,0.463


In [8]:
pd.read_csv(f'./fpl_team_ids_{gw}.csv')

,team_a,team_h,h_fdr,a_fdr,team_home,team_away
0,13,16,4,4,Nottingham Forest,Manchester City
1,9,5,3,3,Brighton,Fulham
2,10,7,2,3,Crystal Palace,Ipswich
3,17,12,2,5,Liverpool,Southampton
4,2,4,3,3,Brentford,Aston Villa
5,8,20,2,2,Wolverhampton Wanderers,Everton
6,11,6,2,4,Chelsea,Leicester
7,3,18,3,3,Tottenham,Bournemouth
8,1,14,4,3,Manchester United,Arsenal
9,15,19,3,2,West Ham,Newcastle United


In [9]:
# Add fdr
odds_ = pd.read_csv(f'./odds_{gw}.csv')
teams = pd.read_csv(f'./fpl_team_ids_{gw}.csv')

def add_fdr(row):
    team_row = teams[teams['team_home'] == row['h_team']]
    h_fdr = team_row['h_fdr'].values[0] if team_row['h_fdr'].values.size > 0 else 'None'
    a_fdr = team_row['a_fdr'].values[0] if team_row['a_fdr'].values.size > 0 else 'None'

    return pd.Series([h_fdr, a_fdr])

odds_[['h_fdr', 'a_fdr']] = odds_.apply(add_fdr, axis=1)
odds_.to_csv(f'./odds_{gw}.csv', index=False)
odds_

,h_team,a_team,WHH,WHD,WHA,h_fdr,a_fdr
0,Nottingham Forest,Manchester City,0.269,0.248,0.483,4,4
1,Brighton,Fulham,0.470,0.261,0.269,3,3
2,Crystal Palace,Ipswich,0.649,0.208,0.144,2,3
3,Liverpool,Southampton,0.863,0.087,0.050,2,5
4,Brentford,Aston Villa,0.400,0.254,0.346,3,3
5,Wolverhampton Wanderers,Everton,0.394,0.291,0.315,2,2
6,Chelsea,Leicester,0.783,0.145,0.072,2,4
7,Tottenham,Bournemouth,0.395,0.243,0.362,3,3
8,Manchester United,Arsenal,0.224,0.268,0.508,4,3
9,West Ham,Newcastle United,0.288,0.250,0.463,3,2


In [10]:
pd.read_csv(f'./odds_26.csv')

,h_team,a_team,WHH,WHD,WHA,h_fdr,a_fdr
0,Leicester,Brentford,0.278,0.249,0.473,2,2
1,Everton,Manchester United,0.365,0.306,0.329,3,3
2,Arsenal,West Ham,0.740,0.174,0.087,2,4
3,Bournemouth,Wolverhampton Wanderers,0.585,0.226,0.190,2,3
4,Fulham,Crystal Palace,0.464,0.272,0.264,3,3
5,Ipswich,Tottenham,0.271,0.256,0.473,3,2
6,Southampton,Brighton,0.190,0.226,0.585,3,2
7,Aston Villa,Chelsea,0.351,0.263,0.386,4,3
8,Newcastle United,Nottingham Forest,0.516,0.248,0.236,3,4
9,Manchester City,Liverpool,0.345,0.243,0.412,5,4


## Update the team names of the odds


In [12]:
prev_odds = pd.read_csv('./E0 24-25.csv')

prev_odds = prev_odds.rename(columns={'HomeTeam': 'h_team', 'AwayTeam': 'a_team'})

def update_name(row):
    # print(row['h_team'], row['a_team'])
    if(row['h_team'] == 'Man United'):
        row['h_team'] = 'Manchester United'
    elif(row['a_team'] == 'Man United'):
        row['a_team'] = 'Manchester United'

    if(row['h_team'] == 'Man City'):
        row['h_team'] = 'Manchester City'
    elif(row['a_team'] == 'Man City'):
        row['a_team'] = 'Manchester City'

    if(row['h_team'] == 'Wolves'):
        row['h_team'] = 'Wolverhampton Wanderers'
    elif(row['a_team'] == 'Wolves'):
        row['a_team'] = 'Wolverhampton Wanderers'

    if(row['h_team'] == 'Newcastle'):
        row['h_team'] = 'Newcastle United'
    elif(row['a_team'] == 'Newcastle'):
        row['a_team'] = 'Newcastle United'

    if(row['h_team'] == "Nott'm Forest"):
        row['h_team'] = 'Nottingham Forest'
    elif(row['a_team'] == "Nott'm Forest"):
        row['a_team'] = 'Nottingham Forest'
    return row


prev_odds = prev_odds.apply(update_name, axis=1)

prev_odds.to_csv('./E0 24-25.csv', index=False)
